# Evaluation for the second model

In [ ]:
from pathlib import Path
import numpy as np
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
import sys; sys.path.insert(0, '/mnt/src')

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_trajectory_datasets

In [ ]:
feature_columns = [
    'left_boom_base_yaw_joint', 
    'left_boom_base_pitch_joint',
    'left_boom_main_prismatic_joint',
    'left_boom_second_roll_joint',
    'left_boom_second_yaw_joint',
    'left_boom_top_pitch_joint',
    'left_boom_ee_joint',
    'cable1_property(length,youngsmodule(bend,twist))',
    'cable2_property(length,youngsmodule(bend,twist))',
    'cable3_property(length,youngsmodule(bend,twist))'
]

label_features = [
    ('cable1_lowest_point', np.array([1], dtype=np.int64)),
    ('cable2_lowest_point', np.array([1], dtype=np.int64)),
    ('cable3_lowest_point', np.array([1], dtype=np.int64))
]

normalized_features = [
    ('cable1_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable2_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable3_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64))
]

In [ ]:
data_folder = Path("/mnt/data/")
train_set, validation_set, test_set, visualization_set = read_trajectory_datasets(data_folder, 0.8, 0.15, 0.045, 0.005, 64, 
                                                                                  feature_columns=feature_columns, label_features=label_features, normalized_features=normalized_features)

In [ ]:
input_shape, output_shape = 16, 3
print(f"Data shape {input_shape} / {output_shape} of total {len(train_set) + len(validation_set) + len(test_set) + len(visualization_set)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
from utils.file_io import define_dataloader_from_subset
import os
import ast
from dotenv import load_dotenv

In [ ]:
model_path = Path("/mnt/models/fully_connected/").absolute()

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
optimizer = os.getenv("OPTIMIZER")
n_layers = int(os.getenv("N_LAYERS"))
hidden_layers_proj_size = ast.literal_eval(os.getenv("HIDDEN_LAYERS_PROJ_SIZE"))
lstm_dropout = float(os.getenv("LSTM_DROPOUT"))
final_dropout = float(os.getenv("FINAL_DROPOUT"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_subset(train_set, validation_set, test_set, batch_size=batch_size)

## Load and Train the model

In [ ]:
from models.lstm import DecoderLSTM, train
from utils.loss_functions import get_loss_function
from utils.optimizer import get_optimizer_function

In [ ]:
model = DecoderLSTM(input_shape, hidden_layers_proj_size[0], output_shape, lstm_dropout, 
                    final_dropout, n_layers, hidden_layers_proj_size[1])
optimizer = get_optimizer_function(optimizer, model, learning_rate)
loss_function = get_loss_function()

In [ ]:
_ = train(num_epochs, train_dataloader, validation_dataloader, model, loss_function, optimizer, model_path, device)

## Evaluation

In [ ]:
from utils.evaluation import compute_losses_from, compute_sliding_window_predictions
from utils.visualization import create_plot_for_dimensions

In [ ]:
model_state_dict = torch.load(model_path / "checkpoint.pt")
model.load_state_dict(model_state_dict)

In [ ]:
model.eval()

## 1. Analysis: Compute test loss

In [ ]:
y, y_true = compute_sliding_window_predictions(test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, get_loss_function())
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")

## 2. Analysis: Show Cable lowest points

In [ ]:
plot = create_plot_for_dimensions(y.numpy(), y_true.numpy(), size=5)
plot.show()